In [1]:
import os
import numpy as np
import torch
import pickle
import time

import torch
from torch.utils.data import DataLoader, Dataset
from models import Pose2AudioTransformer
from transformers import EncodecModel
from utils import DanceToMusic
from datetime import datetime
from torch.optim import Adam

In [2]:
# assign GPU or CPU
# if torch.backends.mps.is_available():
    # device = torch.device("mps")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
# device = torch.device("cpu")

model_id = "facebook/encodec_24khz"
encodec_model = EncodecModel.from_pretrained(model_id)
encodec_model.to(device)
codebook_size = encodec_model.quantizer.codebook_size
sample_rate = 24000

data_dir = "/home/azeez/azeez_exd/misc/DanceToMusic/data/samples"
dataset = DanceToMusic(data_dir, encoder = encodec_model, sample_rate = sample_rate, device=device)

In [3]:
src_pad_idx = 0
trg_pad_idx = 0
learned_weights = '/home/azeez/azeez_exd/misc/DanceToMusic/weights/best_model_0.0194_copy.pt' 
# device = torch.device("mps")
embed_size = dataset.data['poses'].shape[2] * dataset.data['poses'].shape[3]
pose_model = Pose2AudioTransformer(codebook_size, src_pad_idx, trg_pad_idx, device=device, num_layers=4, heads = 8, embed_size=embed_size)
pose_model.load_state_dict(torch.load(learned_weights, map_location=device))
pose_model.to(device)

Pose2AudioTransformer(
  (encoder): Encoder(
    (position_embedding): Embedding(2000, 96)
    (layers): ModuleList(
      (0): TransformerBlock(
        (attention): SelfAttention(
          (values): Linear(in_features=12, out_features=12, bias=False)
          (keys): Linear(in_features=12, out_features=12, bias=False)
          (queries): Linear(in_features=12, out_features=12, bias=False)
          (fc_out): Linear(in_features=96, out_features=96, bias=True)
        )
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): ReLU()
          (2): Linear(in_features=384, out_features=96, bias=True)
        )
        (dropout): Dropout(p=0, inplace=False)
      )
      (1): TransformerBlock(
        (attention): SelfAttention(
          (values): Linear(in_features=12, out_features=12, 

In [ ]:
audio_codes, pose, pose_mask, wav, wav_mask, _, _ = dataset[0]
output = pose_model.generate(pose.unsqueeze(0).to(device), pose_mask.to(device), max_length = 1528)

IndexError: index 1 is out of bounds for dimension 1 with size 1

In [5]:
for i in range(5):
    audio_codes, pose, pose_mask, wav, wav_mask, _, _ = dataset[i]
    sample = pose_model.generate(pose.unsqueeze(0).to(device), pose_mask.to(device), max_length = 100, temperature=5)
    print(sample)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]], device='cuda:0')
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0,

In [85]:
from IPython.display import Audio, display


def audioCodeToWav(audio_code, encodec_model, sample_rate = 24000):
    audio_code = audio_code.reshape(1,1,2,int(len(audio_code)/2))
    audio_code = audio_code.to('cpu')
    audio_scale = [None]
    wav = encodec_model.decode(audio_code, audio_scale)
    return wav

encodec_model = encodec_model.to('cpu')
wav = audioCodeToWav(output[0], encodec_model, sample_rate = 24000)['audio_values']
print(wav.shape)
display(Audio(wav[0][0].detach().numpy(), rate=24000))

torch.Size([1, 1, 32000])


In [82]:
output[0].shape, audio_codes[0].shape

(torch.Size([60]), torch.Size([2, 759]))

In [83]:
output[0], audio_codes[0][0]

(tensor([401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401,
         401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401,
         401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401,
         401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401, 401,
         401, 401, 401, 401], device='mps:0'),
 tensor([ 121,  395,  537,  537,  662,  401,   34,  568,  844,  572,  231,  758,
          715,  637,  790,  568,  446,  657, 1021,  657,  419,  713,  322,  568,
          568,  568,  924,  560,  713,  384,  445,  754,  509,  362,  568,  434,
          797,  352,  246,  189,  568,  713,  659,  568,  568,  568,  568,  659,
          560,  169,  560,  701,  788,  659,  817,  437,  560,  531,  560,  782,
          568,  568,  568,  560,  543,  654,  631,  152,  152,  715,  388,  388,
          388,  366,  844,  568,  388,  388,  388,  388,  213,  213,  213,  560,
          388,  388,  659,  790,  830,  713, 1021,  790,  322,  560,  